### Import packages

In [1]:
from google.cloud import storage
from google.cloud import bigquery
import numpy as np
import string

### Setup connection

In [2]:
REGION = 'us-central1'
#Get Project ID
project = !gcloud config get-value project
PROJECT_ID = project[0]
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'yelp'
bq_dir = BQ_PROJECT + '.'+BQ_DATASET
print('BigQuery project ID:', BQ_PROJECT)
print('BigQuery dataset:', BQ_DATASET)
print('BiqQuery dataset directory:', bq_dir)
#Set clients:
gcs = storage.Client(project = PROJECT_ID)
bq = bigquery.Client(project = PROJECT_ID)


BigQuery project ID: valiant-song-371916
BigQuery dataset: yelp
BiqQuery dataset directory: valiant-song-371916.yelp


In [3]:
### Get the list of tables from the dataset yelp:
dataset_id = BQ_DATASET
tables = bq.list_tables(dataset_id)  # Make an API request.

print("Tables contained in '{}':".format(dataset_id))
for table in tables:
    print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))


Tables contained in 'yelp':
valiant-song-371916.yelp.business
valiant-song-371916.yelp.checkin
valiant-song-371916.yelp.reviews
valiant-song-371916.yelp.tip
valiant-song-371916.yelp.user


In [4]:
bq_dir = BQ_PROJECT + '.'+BQ_DATASET
bq_dir

'valiant-song-371916.yelp'

### Data Exploration

In [5]:
### Get the first 5 instances from business table:
query = f"""
            SELECT *
            FROM {bq_dir}.business
            LIMIT 5
    """
bq_cm = bq.query(query = query).to_dataframe()
bq_cm

,categories,attributes,state,is_open,postal_code,name,review_count,hours,stars,latitude,address,longitude,city,business_id
0,"Real Estate Agents, Home Services, Real Estate",None,IN,1,46032,Jennie Deckert,7,None,5.0,39.969300,,-86.165002,Carmel,REwfwz-_-CMQ7Np5UVi9Qg
1,"Banks & Credit Unions, Financial Services",None,DE,1,19850,Chase JP Morgan Bank Credit Card Services,111,None,1.5,39.749361,,-75.643331,Wilmington,7PDi_iyik3jraDAzWwwR4Q
2,"Couriers & Delivery Services, Local Services, ...",None,TN,1,37027,Nashville Delivers,7,None,3.5,36.002397,6591 Bluff Rd,-86.702381,Brentwood,qFX8IdomeBVhDPCSyz2aRg
3,"Banks & Credit Unions, Financial Services",None,PA,1,19083,TD Bank Havertown,6,None,1.5,39.985661,120 W Eagle Rd,-75.314844,Haverford,FC6ef4rMMZKNov_-A9M6iQ
4,"Gas Stations, Automotive, Towing",None,NJ,1,08057,Lott's Service,8,None,1.5,39.973753,"908 N Lenola Rd, Ste 3",-74.995949,Moorestown,DcUDIlxFsvqPvFh8_lb4Sw


In [6]:
### Get the 5 most popular categories in yelp reviews
query = f"""
            SELECT categories, count(*) as total_reviews
            FROM {bq_dir}.business
            GROUP BY categories
            ORDER BY total_reviews DESC
    """
cat_df = bq.query(query = query).to_dataframe()
print('Df shape:', cat_df.shape)
cat_df.head()

Df shape: (83143, 2)


,categories,total_reviews
0,"Beauty & Spas, Nail Salons",1012
1,"Restaurants, Pizza",935
2,"Nail Salons, Beauty & Spas",934
3,"Pizza, Restaurants",821
4,"Restaurants, Mexican",727


In [8]:
cat_df['categories'].tail()

83138    Fitness & Instruction, Active Life, Arts & Ent...
83139    Tapas Bars, Bars, Restaurants, Nightlife, Japa...
83140    Barre Classes, Active Life, Cycling Classes, F...
83141    Kids Activities, Skating Rinks, Recreation Cen...
83142    Thrift Stores, Antiques, Furniture Stores, Hom...
Name: categories, dtype: object

The categories from yelp dataset was not fully defined with more than 80000 categories or combined multiple categories, which could be due to the keywords for each business. 

Thus, we will need to build a model to classify these labels into different categories. Particularly, we will build a K-means clustering model for this specific tasks (See the section below)

### K-means clustering model to determine the business categories.

In [9]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
#from sklearn.metrics.scorer import SCORERS

#### Prepare data for K-means clustering

In [10]:
# Define the categories after clean up:
def split_and_replace(word):
    return str(word).replace('&','and').split(',')

categories = cat_df['categories'].apply(lambda x: split_and_replace(x))

# Convert the categories to a list of strings
categories = [" ".join(category) for category in categories]
print(categories[0:5])

['Beauty and Spas  Nail Salons', 'Restaurants  Pizza', 'Nail Salons  Beauty and Spas', 'Pizza  Restaurants', 'Restaurants  Mexican']


#### Text Normalization
Here we will convert text to numerical features using CountVectorizer() and implement two methods for text normalization, which includes the StandardScaler() to remove the mean and scaling to unit variance, and the TF-IDF transformer for text normalization. Performance of KMeans with different normalizer is evaluated

In [12]:
# Create a CountVectorizer to transform the categories into numerical features
vectorizer = CountVectorizer()
X_vect = vectorizer.fit_transform(categories)

#Normalization
scaler = StandardScaler(with_mean = False)
X_normed = scaler.fit_transform(X_vect)

#Transform to TF-IDF
transformer = TfidfTransformer()
X_tfdf = transformer.fit_transform(X_vect)

In [16]:
### Check the kmean model with non-normalized dataset
model = KMeans(n_clusters=20)
model.fit(X_vect.toarray())

cluster_assigment = model.predict(X_vect.toarray())

# Evaluate the model using various metrics
silhouette = silhouette_score(X_vect, cluster_assigment)
print(silhouette)

0.08512171340727473


In [13]:
### Check the kmean model with normalized dataset:
model_normed = KMeans(n_clusters=20)
model_normed.fit(X_normed.toarray())

cluster_assigment_normed = model_normed.predict(X_normed.toarray())

# Evaluate the model using various metrics
silhouette = silhouette_score(X_normed, cluster_assigment_normed)
print(silhouette)

-0.11983316966146747


In [14]:
### Check the kmean model with tf-idf dataset:
model_tfdf = KMeans(n_clusters=20)
model_tfdf.fit(X_tfdf.toarray())

cluster_assigment_tfdf = model_tfdf.predict(X_tfdf.toarray())

# Evaluate the model using various metrics
silhouette = silhouette_score(X_tfdf, cluster_assigment_tfdf)
print(silhouette)

0.09690581100858855


**Based on the silhouette score, we will choose the tfdf transformer as a method for text normalization**

#### Perform hyperparameters tuning

In [17]:
# Define the hyperparameters to tune and their possible values
param_grid = {
    "n_clusters": [40,60,80,100],
    "init": ["k-means++"],
    #"max_iter": [300, 500, 1000],
    #"tol": [1e-4, 1e-3, 1e-2]
}

# Define a scorer based on silhouette score for GridSearchCV
def scorer(estimator, X, y=None):
    y_pred = estimator.fit_predict(X)
    return silhouette_score(X, y_pred)

# Create a KMeans model
kmeans = KMeans()

# Create a GridSearchCV object to perform hyperparameter tuning
kmeans_tuning = GridSearchCV(kmeans, param_grid, cv=5,
                           scoring=scorer,
                           verbose=1)

# Fit the GridSearchCV object to the data
kmeans_tuning.fit(X_tfdf)

# Get the best parameters from the grid search
best_params = kmeans_tuning.best_params_

# Print the best parameters
print(f"Best parameters: {best_params}")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameters: {'init': 'k-means++', 'n_clusters': 100}


In [127]:
def get_category(samples, vectorizer, transformer, eval_model):
      
    new_samples = [" ".join(word) for word in samples]
    
    # Transform the new category into numerical features
    new_samples = transformer.transform(vectorizer.transform(new_samples))

    # Get the prediction for the new category
    prediction = eval_model.best_estimator_.predict(new_samples)
    return prediction

#Preprocess raw data
categories = cat_df['categories'].apply(lambda x: split_and_replace(x))

#Get predictions
cat_df['predict_cat'] = get_category(categories, vectorizer, 
                                     transformer, kmeans_tuning)


In [141]:
# Define the hyperparameters to tune and their possible values
param_grid = {
    "n_clusters": [100],
    "init": ["k-means++"],
    #"max_iter": [300, 500, 1000],
    #"tol": [1e-4, 1e-3, 1e-2]
}

# Define a scorer based on silhouette score for GridSearchCV
def scorer(estimator, X, y=None):
    y_pred = estimator.fit_predict(X)
    return silhouette_score(X, y_pred)

# Create a KMeans model
kmeans = KMeans()

# Create a GridSearchCV object to perform hyperparameter tuning
kmeans_tuning = GridSearchCV(kmeans, param_grid, cv=5,
                           scoring=scorer,
                           verbose=3)

# Fit the GridSearchCV object to the data
kmeans_tuning.fit(X_vect)

# Get the best parameters from the grid search
best_params = kmeans_tuning.best_params_

# Print the best parameters
print(f"Best parameters: {best_params}")

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END ....init=k-means++, n_clusters=100;, score=0.166 total time= 1.4min
[CV 2/5] END ....init=k-means++, n_clusters=100;, score=0.159 total time= 1.6min
[CV 3/5] END ....init=k-means++, n_clusters=100;, score=0.159 total time= 1.5min
[CV 4/5] END ....init=k-means++, n_clusters=100;, score=0.127 total time= 1.7min
[CV 5/5] END ....init=k-means++, n_clusters=100;, score=0.111 total time= 1.5min
Best parameters: {'init': 'k-means++', 'n_clusters': 100}


In [18]:
#Preprocess raw data
categories = cat_df['categories'].apply(lambda x: split_and_replace(x))
categories = [" ".join(category) for category in categories]

X_vect = vectorizer.transform(categories)

#Get predictions
cat_df['predict_cat'] = kmeans_tuning.best_estimator_.predict(X_vect)


In [19]:
cat_df['predict_cat'].value_counts()

63    2680
94    1954
32    1856
98    1752
7     1739
      ... 
53     266
8      241
97     235
51     166
11     135
Name: predict_cat, Length: 100, dtype: int64

In [46]:
print(cat_df[cat_df['predict_cat']==7]['categories'][0:10].to_string())

8                   Food, Coffee & Tea
11                  Coffee & Tea, Food
138         Food, Donuts, Coffee & Tea
149         Donuts, Coffee & Tea, Food
177         Donuts, Food, Coffee & Tea
179         Food, Coffee & Tea, Donuts
186         Coffee & Tea, Food, Donuts
187         Coffee & Tea, Donuts, Food
284    Restaurants, Coffee & Tea, Food
358    Food, Coffee & Tea, Restaurants


#### Save model

In [35]:
import pickle
filename = 'kmeans_model.sav'
pickle.dump(kmeans_tuning, open(filename, 'wb'))